## Example 5: Extractive Distillation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from thermo_models.RaoultsLawModel import *
from distillation.DistillationModel import DistillationModel
import utils.AntoineEquation as AE
from IPython.display import clear_output
from tqdm import tqdm
from distillation.DistillationDoubleFeed import *
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive, Button, VBox, HBox, Output
from IPython.display import clear_output
from thermo_models.VanLaarModel import *
from distillation.residue_curves import *

In [2]:
#Acetone (1 - Light) Methanol (2 - Intermediate) Water (3 - Heavy)
#Table A.6 in Knapp 
A_ij = {
    (1,1):0,
    (1,2):182.0,
    (1,3):795.0,
    (2,1):196,
    (2,2):0,
    (2,3):332.6,
    (3,1):490.0,
    (3,2):163.80,
    (3,3):0
}

#Different definition of Antoine where we have to take the negative of B
Acet_A = 21.3099; Acet_B = 2801.53; Acet_C = -42.875
Meth_A = 23.4832; Meth_B = 3634.01; Meth_C = -33.768
#Assuming P < 2 atm
Water_A = 23.2256; Water_B = 3835.18; Water_C = -45.343

#Knapp Thesis Figure 3.8 uses ln form of Antoine
AcetoneAntoine = AE.AntoineEquationBaseE(Acet_A,Acet_B,Acet_C)
MethanolAntoine = AE.AntoineEquationBaseE(Meth_A, Meth_B, Meth_C)
WaterAntoine = AE.AntoineEquationBaseE(Water_A,Water_B,Water_C)
        
P_sys = 101325
# Create a Raoult's law object
AceMethWaterSys = VanLaarModel(num_comp = 3, P_sys = P_sys, A_coeff = A_ij, comp_names = ["Acetone","Methanol","Water"], partial_pressure_eqs = [AcetoneAntoine, MethanolAntoine, WaterAntoine])


In [3]:
#Ethanol (1 - Light) Water (2 - Intermediate) Ethylene Glycol (3 - Heavy)
#Table A.5 in Knapp, assuming P < 2 atm
A_ij_EWEG = {
    (1,1):0,
    (1,2):615.9,
    (1,3):277.3,
    (2,1):314,
    (2,2):0,
    (2,3):376.028,
    (3,1):892.7,
    (3,2):622.539,
    (3,3):0
}

#Different definition of Antoine where we have to take the negative of B
#Table A.4 in Knapp
#Assuming P < 2 atm
Eth_A = 23.5807; Eth_B = 3673.81; Eth_C = -46.681
Water_A = 23.2256; Water_B = 3835.18; Water_C = -45.343
EG_A = 25.1431; EG_B = 6022.18; EG_C = -28.25


#Knapp Thesis Figure 3.8 uses ln form of Antoine
EthanolAntoine = AE.AntoineEquationBaseE(Eth_A, Eth_B, Eth_C)
WaterAntoine = AE.AntoineEquationBaseE(Water_A, Water_B, Water_C)
EthyleneGlycolAntoine = AE.AntoineEquationBaseE(EG_A, EG_B, EG_C)
        
P_sys = 101325
# Create a Raoult's law object
EthWaterEGSys = VanLaarModel(num_comp = 3, P_sys = P_sys, A_coeff = A_ij_EWEG, comp_names = ["Ethanol","Water","Ethylene Glycol"], partial_pressure_eqs = [EthanolAntoine, WaterAntoine, EthyleneGlycolAntoine])



In [ ]:
out_residue = Output()
rcm = residue_curve(AceMethWaterSys)
fig, ax = plt.subplots(1,1,figsize= (7,7))
rcm.plot_residue_curve_int(ax, [0,7],data_points = 100, 
                            init_comps = [ 
                                            np.array([0.4,0.4,0.2]),
                                            np.array([0.6,0.2,0.2]),
                                            np.array([0.6,0.1,0.3]),
                                            np.array([0.1,0.8,0.1]),
                                            np.array([0.1,0.1,0.8])
                                            ])
plt.show()

#### Demo 2

In [16]:
Fr = 1.0
xFL = np.array([0.3, 0.6, 0.1])
xFU = np.array([0.2, 0.1, 0.7])
zF = (Fr/(1+Fr))*xFU + (1/(1+Fr))*xFL                                #Knapp Equation 5.9
xD = np.array([0.93, 0.04, 0.03]) 
D_B = ((zF[0] - 0.02) / (xD[0] - zF[0]))                             #Knapp Equation 5.13, set xB[0] = 0.01
xB = np.array([0.02, (zF[1] - D_B*(xD[1]-zF[1])), (zF[2] - D_B*(xD[2]*zF[2]))])

R = 3
distillation_model = DistillationModelDoubleFeed(AceMethWaterSys, Fr = Fr, zF = zF, xFL = xFL, xFU = xFU, xD = xD, xB = xB, reflux = R)

# Create an interactive plot function
def interactive_plot(middle_start):
    fig, ax = plt.subplots(1, 1, figsize=(5, 10))
    distillation_model.plot_rect_strip_comp(ax, middle_start=middle_start)
    plt.show()

# Create a dropdown widget for middle_start
middle_start_widget = widgets.Dropdown(
    options=[2, 4, 6, 8, 10],
    description='Middle Section Start:',
    disabled=False
)

# Create an interactive plot with the dropdown widget
interactive(interactive_plot, middle_start=middle_start_widget)

interactive(children=(Dropdown(description='Middle Section Start:', options=(2, 4, 6, 8, 10), value=2), Output…

### Demo 3
A maximum reflux ration exists, around R=60?

In [18]:
R_slider =  widgets.FloatSlider(value=0.1, min=0, max=100, step=0.001,  description='Reflux Ratio:', readout_format='.1f')
go_button = widgets.Button(description='Create Model', width='15%', margin='0 10px 0 0')
system_widget = widgets.Dropdown(
    options=["Acetone/Methanol/Water","Ethanol/Water/Ethylene Glycol"],
    description='Choice of Components:',
    disabled=False)
out_middle = Output()

def plot_reflux_demo_ex5(R = R_slider, thermo_system = system_widget.value):
    thermo_choice = AceMethWaterSys
    if thermo_system == 'Ethanol/Water/Ethylene Glycol':
        thermo_choice = EthWaterEGSys
    plt.close('all')  # Ensure that any existing figures are closed before plotting a new one.
    Fr = 1.0
    xFL = np.array([0.3, 0.6, 0.1])
    xFU = np.array([0.2, 0.1, 0.7])
    zF = (Fr/(1+Fr))*xFU + (1/(1+Fr))*xFL                                #Knapp Equation 5.9
    xD = np.array([0.93, 0.04, 0.03]) 
    D_B = ((zF[0] - 0.02) / (xD[0] - zF[0]))                             #Knapp Equation 5.13, set xB[0] = 0.01
    xB = np.array([0.02, (zF[1] - D_B*(xD[1]-zF[1])), (zF[2] - D_B*(xD[2]*zF[2]))])

    middle_start = 6
    distillation_model = DistillationModelDoubleFeed(thermo_model = thermo_choice, Fr = Fr, zF = zF, xFL = xFL, xFU = xFU, xD = xD, xB = xB, reflux = R)
    fig, ax = plt.subplots(1,1,figsize= (5,10))
    distillation_model.plot_rect_strip_comp(ax, middle_start=middle_start)
    plt.subplots_adjust(hspace=0)
    plt.show()

def on_go_button_click(b):
    with out_middle:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_reflux_demo_ex5(R=R_slider.value, thermo_system=system_widget.value)
        
go_button = Button(description='Generate Plot')
go_button.on_click(on_go_button_click)

layout = VBox([system_widget, HBox([R_slider, go_button]), out_middle])
display(layout)


### Demo 4
Changing Feed Ratio

In [ ]:
Fr_slider =  widgets.FloatSlider(value=0.64, min=0, max=5, step=0.001,  description='Feed Ratio:', readout_format='.2f')
go_button = widgets.Button(description='Create Model', width='15%', margin='0 10px 0 0')
out_middle = Output()

def plot_reflux_demo_ex5(Fr = Fr_slider):
    plt.close('all')  # Ensure that any existing figures are closed before plotting a new one
    xFL = np.array([0.3, 0.6, 0.1])
    xFU = np.array([0.2, 0.1, 0.7])
    zF = (Fr/(1+Fr))*xFU + (1/(1+Fr))*xFL                                #Knapp Equation 5.9
    xD = np.array([0.95, 0.04, 0.01]) 
    D_B = ((zF[0] - 0.01) / (xD[0] - zF[0]))                             #Knapp Equation 5.13, set xB[0] = 0.01
    xB = np.array([0.01, (zF[1] - D_B*(xD[1]-zF[1])), (zF[2] - D_B*(xD[2]*zF[2]))])

    R = 3
    middle_start = 6
    distillation_model = DistillationModelDoubleFeed(AceMethWaterSys, Fr = Fr, zF = zF, xFL = xFL, xFU = xFU, xD = xD, xB = xB, reflux = R)
    fig, ax = plt.subplots(1,1,figsize= (5,10))
    distillation_model.plot_rect_strip_comp(ax, middle_start=middle_start)
    plt.subplots_adjust(hspace=0)
    plt.show()


def on_go_button_click(b):
    with out_middle:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_reflux_demo_ex5(Fr=Fr_slider.value)
        
go_button = Button(description='Generate Plot')
go_button.on_click(on_go_button_click)

layout = VBox([HBox([Fr_slider, go_button]), out_middle])
display(layout)